<a href="https://colab.research.google.com/github/leeCodingStudio/Python_NLP/blob/master/13_GPT3%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%83%9D%EC%84%B1_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.7 MB/s eta 0:00:00


In [2]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 11.9 MB/s eta 0:00:00


* [KaKaoBrain KoGPT 예제](https://velog.io/@gtpgg1013/kogpt-%ED%95%9C%EA%B5%AD%EC%96%B4-%EC%83%9D%EC%84%B1-GPT-3-%EB%AA%85%EB%AC%B8%EA%B0%80-%EB%82%A9%EC%8B%9C%EC%98%A4)
* [GPT 디코딩 전략](https://littlefoxdiary.tistory.com/46)

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTJForCausalLM

In [4]:
tokenizer = AutoTokenizer.from_pretrained('MrBananaHuman/kogpt_6b_fp16',
                                          bos_token='[BOS]', # 시작
                                          eos_token='[EOS]', # 끝
                                          unk_token='[UNK]',
                                          pad_token='[PAD]',
                                          mask_token='[MASK]')

In [ ]:
model = GPTJForCausalLM.from_pretrained('MrBananaHuman/kogpt_6b_fp16',
                                        pad_token_id=tokenizer.eos_token_id,
                                        torch_dtype='auto',
                                        low_cpu_mem_usage=True)
model.to('cuda', non_blocking=True)
model.eval()

In [ ]:
# Beam Search 기반

prompt = '야! 너는 AI지? 정말로 말을 알아듣니?'
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(
        tokens,
        max_length=100,
        num_beams=5, # 1보다 큰 값을 지정
        no_repeat_ngram_size=2, # 2-gram의 어구가 반복되지 않도록 설정함
        num_return_sequences=5, # 5개의 문장을 리턴
        early_stopping=True # EOS 토큰이 나오면 생성을 중단
    )
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)

In [ ]:
# Sampling 기반

prompt = '야! 너는 AI지? 정말로 말을 알아듣니?'
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(
        tokens,
        max_length=100,
        do_sample=True, # 샘플링 전략을 사용
        top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플리에서 제외
        top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
        temperature=0.75, # 낮으면 기존 단어를 재활용 확률이 높고, 높으면 random 확률이 감소
        num_return_sequences=3 # 3개의 결과를 디코딩함
    )
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)

* 문장을 생성할 때 beam search보다 sample 기반이 좋은 전략일 수 있음
* top-k / top-p 샘플링 전략에서도 동어 반복 문제는 생길 수 있음
* 다양한 방법을 실험해보며 사용자의 task에 적합한 전략을 선택하는 것이 중요

In [ ]:
prompt = '''
한국어: "나 지금 많이 배고픈데, 우리 여기서 뭐라도 먹고 갈까?"
영어: ""
'''
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(
        tokens,
        max_length=100,
        do_sample=True, # 샘플링 전략을 사용
        top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플리에서 제외
        top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
        temperature=0.75, # 낮으면 기존 단어를 재활용 확률이 높고, 높으면 random 확률이 감소
        num_return_sequences=3 # 3개의 결과를 디코딩함
    )
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)

In [ ]:
prompt = '''
요즘 굉장히 핫한 인공지능인 GPT님과 인터뷰를 나눠보겠습니다!
Q : GPT님, 요즘 바쁘시죠?
A : '''
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(
        tokens,
        max_length=100,
        do_sample=True, # 샘플링 전략을 사용
        top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플리에서 제외
        top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
        temperature=0.75, # 낮으면 기존 단어를 재활용 확률이 높고, 높으면 random 확률이 감소
        num_return_sequences=3 # 3개의 결과를 디코딩함
    )
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)

In [ ]:
prompt = '''
요즘 굉장히 핫한 인공지능인 GPT님과 인터뷰를 나눠보겠습니다!
Q : GPT님, 요즘 바쁘시죠?
A : '''
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(
        tokens,
        max_length=256,
        do_sample=True, # 샘플링 전략을 사용
        top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플리에서 제외
        top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
        temperature=0.85, # 낮으면 기존 단어를 재활용 확률이 높고, 높으면 random 확률이 감소
        num_return_sequences=3 # 3개의 결과를 디코딩함
    )
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)